## 欢迎进入 ModelWhale Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [14]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from pyecharts import options as opts
from pyecharts.charts import HeatMap
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Map

In [11]:
#读取数据，进行数据清洗
#读取全球年度地表温度变化数据
# 读取原始数据csv文件

df = pd.read_csv('/home/mw/input/temp6990/Temp.csv')

# 去除不需要的列
df.drop(['ObjectId', 'Country/Region','ISO2','Indicator', 'Unit', 'Source','CTS_Code','CTS_Name','CTS_Full_Descriptor'], axis=1, inplace=True)


# 先计算每列的平均值
mean_values = df.mean()

# 使用平均值来填充缺失值
df = df.fillna(mean_values)

# 接下来，可以通过均值、方差等统计量来识别和去除异常值。

# 下面是一种基于均值和标准差的简单方法：
def remove_outliers(df, column, threshold):
    # 计算该列的均值和标准差
    mean_value = df[column].mean()
    std_value = df[column].std()
    # 找出超过阈值的异常数据的索引
    outliers = abs(df[column] - mean_value) > threshold * std_value
    # 删除这些异常数据所在的行
    df = df.drop(df[outliers].index)
    return df

# 将第一列国家代码作为索引列。
df = df.set_index(df.columns[0])

# 对每一列数据进行异常值检测并删除异常值
for col in df.columns:
    df = remove_outliers(df, col, 3)


df_copy = df.copy()  # 复制数据，避免修改原数据

# 对于每一行数据，从第三列开始，加上前一列的值
for i in range(len(df)):
    row = df_copy.iloc[i, :].copy()
    for j in range(2, len(row)):
        row[j] += row[j-1]
    df.iloc[i, :] = row

#将 DataFrame 保存为 csv 文件
df.to_csv('/home/mw/project/temp_wash.csv', index=True)


In [13]:
#绘制折线图
# 读入数据
df = pd.read_csv('/home/mw/project/temp_wash.csv', index_col='ISO3')

# 绘制折线图
fig, ax = plt.subplots(figsize=(20,10))

# 循环遍历所有国家
for country in df.index:
    # 绘制该国家的折线图
    df.loc[country].plot(kind='line', ax=ax)

# 设置标题、x轴标签、y轴标签
ax.set_title('Temperature of Countries from 1961 to 2021')
ax.set_xlabel('Year')
ax.set_ylabel('Temperature')

# 显示图例
ax.legend(df.index, loc='upper left', ncol=4)

# 显示图像
plt.show()


<Figure size 1440x720 with 1 Axes>

In [12]:
#绘制热力图
# 读取数据集
df = pd.read_csv('/home/mw/project/temp_wash.csv', index_col='ISO3')

# 设置画布大小
plt.figure(figsize=(20, 10))

# 绘制热力图
sns.heatmap(df, cmap='coolwarm', annot=False, fmt='.2f', linewidths=.5)

# 调整字体大小
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# 添加标题
plt.title('Temperature of Countries from 1961 to 2021', fontsize=20)

# 调整标题位置
plt.subplots_adjust(top=0.95)

# 显示图像
plt.show()


<Figure size 1440x720 with 2 Axes>

In [6]:
# 处理国际货币基金组织（IMF）的全球数据
# 读取原始数据文件
df = pd.read_csv('/home/mw/input/imf3505/WEOOct2022all.csv', encoding='utf-8')

# 需要保留的六个指标
# NGDPD：国内生产总值，以当前国际价格计算
# NGDP_RPCH：国内生产总值的实际增长率，以百分比形式表示
# NGDPDPC：人均国内生产总值，按当前国际价格计算
# GGX：政府财政收入净额
# PCPIPCH：消费者物价指数的百分比变化率
# LUR：劳动力失业率，衡量就业人口中没有工作的人数所占比例
keep_indicators = ['NGDP_RPCH', 'NGDPDPC', 'GGX', 'PCPIPCH', 'LUR','NGDPD']

# 获取所有国家/地区的名称列表
countries = df['Country/Region'].unique()

# 新建一个DataFrame，用于存储筛选后的数据
new_df = pd.DataFrame(columns=df.columns)

# 对每个国家进行处理
for country in countries:
    # 获取当前国家的数据
    df_country = df[df['Country/Region'] == country]

    # 删除不需要的行
    df_country.drop(df_country[~df_country['WEO Subject Code'].isin(keep_indicators)].index, inplace=True)

    # 添加到新DataFrame中
    new_df = pd.concat([new_df, df_country])

delete_colum = ['WEO Country/Region Code','Country/Region','Subject Descriptor',
                'Units','Scale','Country/Series-specific Notes','Subject Notes',
                'Estimates Start After','2022','2023','2024','2025','2026','2027']
new_df = new_df.drop(columns=delete_colum) # 使用drop函数删除指定的列，columns参数指定要删除的列名列表
# 将筛选后的数据保存为新文件
new_df.to_csv('/home/mw/project/new_WEO2022data.csv', index=False, encoding='utf-8')

# 得到2021GDP排名前20的国家信息
# 读取CSV文件
data = pd.read_csv('/home/mw/project/new_WEO2022data.csv', thousands=',')


# 将空缺值填充为0
data = data.fillna(0)

# 获取2021年的数据
data_2021 = data[['ISO', 'WEO Subject Code', '2021']]

# 筛选出NGDPD这一类数据
data_2021 = data_2021[data_2021['WEO Subject Code'] == 'NGDPD']

# 将2021列的数据类型转为float
data_2021['2021'] = data_2021['2021'].astype(float)

# 按照2021年的NGDPD数据值进行降序排列，获取前20个国家
top20 = data_2021.sort_values(by='2021', ascending=False).head(20)

# 获取前20个国家的ISO列表
iso_list = top20['ISO'].tolist()
print('iso_list\n',iso_list)
# 筛选出ISO在前20个国家列表中的记录
data_filtered = data[data['ISO'].isin(iso_list)]
print('data_filtered\n',data_filtered)
# 将结果写入CSV文件
data_filtered.to_csv('/home/mw/project/new_WEO2022data.csv', index=False)











/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


iso_list
 ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'IND', 'FRA', 'ITA', 'CAN', 'KOR', 'RUS', 'AUS', 'BRA', 'IRN', 'ESP', 'MEX', 'IDN', 'NLD', 'SAU', 'TUR']
data_filtered
       ISO WEO Subject Code       1980       1981       1982       1983  \
54    AUS        NGDP_RPCH      2.894      4.127      0.063     -0.474   
55    AUS            NGDPD    162.821     188.29     186.93    179.364   
56    AUS          NGDPDPC  11,000.11  12,520.00  12,226.55  11,584.20   
57    AUS          PCPIPCH     10.136      9.488     11.352     10.039   
58    AUS              LUR      6.133      5.783      7.183      9.967   
59    AUS              GGX          0          0          0          0   
144   BRA        NGDP_RPCH       9.19       -4.4      0.596       -3.4   
145   BRA            NGDPD    145.819    167.583    179.166    143.652   
146   BRA          NGDPDPC   1,229.88   1,382.55   1,446.23   1,134.93   
147   BRA          PCPIPCH     90.227    101.737    100.574     135.02   
148   BRA           

In [7]:
# 数据拆分
# 读取csv文件
df = pd.read_csv('/home/mw/project/new_WEO2022data.csv')

# 获取所有不重复的WEO Subject Code
weo_codes = df['WEO Subject Code'].unique()

# 分类并保存为不同文件
for weo_code in weo_codes:
    # 选择有相同WEO Subject Code的行
    sub_df = df[df['WEO Subject Code'] == weo_code]

    # 仅删除WEO Subject Code那一列，保留ISO
    sub_df.drop('WEO Subject Code', axis=1, inplace=True)

    # 将空值替换成空格
    sub_df.fillna('', inplace=True)

    # 将结果保存到csv文件中，文件名为WEO Subject Code.csv
    #sub_df.to_csv(f'{weo_code}.csv', index=False)
    sub_df.to_csv(f'/home/mw/project/{weo_code}.csv', index=False)



/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [15]:
# 读取数据
df = pd.read_csv("/home/mw/project/LUR.csv", index_col=0)

# 统计每一行缺失值的数量
missing_counts = df.isna().sum(axis=1)

# 删除缺失值过多的行
df = df[missing_counts <= 10]

# 用均值填充缺失值
df = df.fillna(df.mean())

# 转换数据格式
data = []
for index, row in df.iterrows():
    for col_name, value in zip(df.columns, row.values[::-1]):
        data.append([col_name, index, value])

# 计算颜色区间和步长
max_val = df.values.max()
min_val = df.values.min()
step = (max_val - min_val) / 4

# 生成热力图
heatmap = (
    HeatMap()
    .add_xaxis(df.columns.tolist())
    .add_yaxis("", df.index.tolist(), data)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="LUR 热力图"),
        visualmap_opts=opts.VisualMapOpts(
            orient="vertical",
            max_=max_val,
            min_=min_val,
            range_color=["#006699", "#7DB9DE", "#F88F70", "#E0312E"], # 深蓝色、天蓝色、浅红色、红色
            split_number=4 # 分为四段
        ),
        xaxis_opts=opts.AxisOpts(name="年份", type_="category"),
        yaxis_opts=opts.AxisOpts(name="国家名称", type_="category")
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)

# 显示热力图
heatmap.render_notebook()




In [16]:
# 读取文件
df = pd.read_csv('/home/mw/project/NGDPD.csv', thousands=',')

# 将0替换为缺失值
df.replace(0, np.nan, inplace=True)

# 统计每行缺失值的数量
missing_values = df.isnull().sum(axis=1)

# 删除缺失值过多的行
df.drop(missing_values[missing_values > 10].index, inplace=True)

# 使用均值填充缺失值
df.fillna(df.mean(), inplace=True)



# 转换为时间序列数据
df.set_index('ISO', inplace=True)
df.index.name = 'Country'
df.columns = pd.to_datetime(df.columns, format='%Y')

# 绘制折线图
plt.figure(figsize=(20,10))
for country in df.index:
    plt.plot(df.columns, df.loc[country], label=country)
plt.legend(title='Country')
plt.xlabel('Year')
plt.ylabel('NGDPD')
plt.title('GDP by Country and Year')
plt.show()


<Figure size 1440x720 with 1 Axes>

In [17]:
# 读取数据
df = pd.read_csv("/home/mw/project/NGDP_RPCH.csv", index_col=0)

# 统计每一行缺失值的数量
missing_counts = df.isna().sum(axis=1)

# 删除缺失值过多的行
df = df[missing_counts <= 10]

# 用均值填充缺失值
df = df.fillna(df.mean())

# 转换数据格式
data = []
for index, row in df.iterrows():
    for col_name, value in zip(df.columns, row.values[::-1]):
        data.append([col_name, index, value])

# 生成热力图
heatmap = (
    HeatMap()
    .add_xaxis(df.columns.tolist())
    .add_yaxis("", df.index.tolist(), data)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="NGDP_RPCH 热力图"),
        visualmap_opts=opts.VisualMapOpts(max_=df.values.max(), min_=df.values.min(), orient="horizontal"),
        xaxis_opts=opts.AxisOpts(name="年份", type_="category"),
        yaxis_opts=opts.AxisOpts(name="国家名称", type_="category")
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)

# 显示热力图
heatmap.render_notebook()


In [ ]:


# 读取数据
data = pd.read_csv('/home/mw/input/china_temper1679/2001_气温.csv',encoding = "gbk")

# 按照省份分组，计算平均气温
province_data = data.groupby('省份')['平均气温'].mean().reset_index()

# 构造地图数据
map_data = [(row['省份'], row['平均气温']) for index, row in province_data.iterrows()]

# 绘制地图
m = Map()
m.add("", map_data, "china")
m.set_global_opts(
    title_opts=opts.TitleOpts(title="201年各省份平均气温"),
    visualmap_opts=opts.VisualMapOpts(max_=30),
)
#m.render("average_temperature.html")
m.render_notebook()


In [23]:
import pandas as pd
from pyecharts.charts import Map
from pyecharts import options as opts


# 读取数据
data = pd.read_csv('/home/mw/input/china_temper1679/2021_气温.csv',encoding = "gbk")

# 按照省份分组，计算平均气温
province_data = data.groupby('省份')['平均气温'].mean().reset_index()

# 构造地图数据
map_data = [(row['省份'], row['平均气温']) for index, row in province_data.iterrows()]

# 绘制地图
m = Map()
m.add("", map_data, "china")
m.set_global_opts(
    title_opts=opts.TitleOpts(title="2021年各省份平均气温"),
    visualmap_opts=opts.VisualMapOpts(max_=30),
)
#m.render("average_temperature.html")
m.render_notebook()


In [16]:
!pip show pyecharts

Name: pyecharts
Version: 1.1.0
Summary: Python options, make charting easier
Home-page: https://github.com/pyecharts/pyecharts
Author: chenjiandongx
Author-email: chenjiandongx@qq.com
License: MIT
Location: /opt/conda/lib/python3.6/site-packages
Requires: jinja2, prettytable
Required-by: 


In [7]:


class TemperatureMap:
    def __init__(self, year):
        self.year = str(year)
        if len(self.year) != 4:
            raise ValueError("Invalid year format! Please input a 4-digit year, e.g. '2021'.")
    
    def generate_map(self):
        # 读取数据
        data = pd.read_csv("/home/mw/input/china_temper1679/"+self.year+"_气温.csv",encoding = "gbk")

        # 按照省份分组，计算平均气温
        province_data = data.groupby('省份')['平均气温'].mean().reset_index()

        # 构造地图数据
        map_data = [(row['省份'], row['平均气温']) for index, row in province_data.iterrows()]

        # 绘制地图
        m = Map()
        m.add("", map_data, "china")
        m.set_global_opts(
            title_opts=opts.TitleOpts(title=f"{self.year}年各省份平均气温"),
            visualmap_opts=opts.VisualMapOpts(max_=30),
        )
        #m.render(f"{self.year}_average_temperature.html")
        #m.render_notebook()
        return m






In [8]:
a = TemperatureMap(2001)
a.generate_map().render_notebook()

In [9]:
a = TemperatureMap(2011)
a.generate_map().render_notebook()


In [10]:
a = TemperatureMap(2021)
a.generate_map().render_notebook()